In [19]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
import numpy as np

In [72]:
file = './merged_file.csv'  # Replace with your actual file path
df = pd.read_csv(file)
print(len(df))
df.head()

20588


,Description,Extension,Category,MIME type
0,MPEG-4 video file format,.mp4,Video,video/mp4
1,H.264/MPEG-4 AVC video data file,0.264,Video,Unknown
2,Youtube for Android video chunk,.exo,Video,Unknown
3,"ShockWave Flash, Animated vector format for th...",.swf,Video,application/x-shockwave-flash
4,AnyDesk session record,.anydesk,Video,Unknown


In [3]:
unique_df = df[['Extension', 'Category']].drop_duplicates()
len(unique_df)

20588

In [35]:
grouped_df = df.groupby('Extension')['Category'].nunique()

# Filter for extensions with more than one category
extensions_with_multiple_categories = grouped_df[grouped_df > 1]

# Print the results
extensions_with_multiple_categories

Extension
.!bt     2
.!ut     2
.###     2
.$$$     4
.$ac     2
        ..
0.6      2
0.602    2
0.7      2
0.8      4
0.9      2
Name: Category, Length: 4873, dtype: int64

## Dictionary Format

In [10]:
extension_dict = {}
for _, row in unique_df.iterrows():
    extension = row['Extension']
    category = row['Category']
    
    if extension not in extension_dict:
        extension_dict[extension] = []
    extension_dict[extension].append(category)

In [11]:
# Remove duplicates in the category lists
for extension in extension_dict:
    extension_dict[extension] = list(set(extension_dict[extension]))

In [37]:
input_extension = input("Enter an extension (e.g., .pdf) ")
categories = extension_dict.get(input_extension, None)
if categories:
    print(f"Categories for '{input_extension}': {', '.join(categories)}")
else:
    print(f"No categories found for '{input_extension}'.")

Enter an extension (e.g., .pdf)  .!bt


Categories for '.!bt': Misc Files, Internet related


## MultiLabelBinarizer

In [75]:
le_ext = LabelEncoder()
df['Extension_encoded'] = le_ext.fit_transform(df['Extension'])

le_mime = LabelEncoder()
df['MIME_encoded'] = le_mime.fit_transform(df['MIME type'])

# le_meta = LabelEncoder()
# df['Metadata_encoded'] = le_meta.fit_transform(df['Description'])

le_category = LabelEncoder()
df['Category_encoded'] = le_category.fit_transform(df['Category'])

In [93]:
df_grouped = df.groupby('Extension')['Category'].apply(list).reset_index()

# Fit and transform categories into a multi-label binary format
y_multilabel = mlb.fit_transform(df_grouped['Category'])

# Prepare the features (extensions)
X = le_ext.transform(df_grouped['Extension']).reshape(-1, 1)

X.shape , y_multilabel.shape

((13444, 1), (13444, 79))

In [94]:
model = MultiOutputClassifier(RandomForestClassifier(random_state=42))
model.fit(X, y_multilabel)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42))

In [95]:
def predict_new_extension(extension):
    # Transform the extension into the encoded form
    ext_encoded = le_ext.transform([extension])

    # Predict multiple categories using the trained model
    prediction = model.predict(ext_encoded.reshape(-1, 1))

    # Inverse transform to get the actual category labels
    predicted_categories = mlb.inverse_transform(prediction)

    print(f"Predicted Categories for '{extension}': {predicted_categories[0]}")

In [100]:
new_extension = '.$$$'
predict_new_extension(new_extension)

Predicted Categories for '.$$$': ('Backup', 'Backup Files', 'System file', 'Temporary')
